In [ ]:
# Import Data handling/display libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from typing import List, Union, Dict
# put this lower down.....
sns.set()
sns.set_context("talk")

# Import scikit-learn core slibraries
from sklearn.metrics import auc, roc_auc_score, roc_curve, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Import IBM's AI Fairness tooolbox
from aif360.datasets import BinaryLabelDataset  # To handle the data
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric  # For calculating metrics
from aif360.explainers import MetricTextExplainer  # For explaining metrics
from aif360.algorithms.preprocessing import Reweighing  # Preprocessing technique

# Warnings will be used to silence various model warnings for tidier output
import warnings
#warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('./input/Taiwan-Credit-Card-Cleaned.csv')
train.head(5)

In [ ]:
X = train.drop("DEFAULT",axis=1)
y = train["DEFAULT"]
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 101, stratify=y)

from collections import Counter
print ('this is y_train Counter ()', Counter(y_train))
print('this is y_test Counter ()', Counter(y_test))
print('this is y_train unique', np.unique(y_train))
print('this is y_test  unique', np.unique(y_test))

In [ ]:
train.GENDER.value_counts()

In [ ]:
# Make males 0, Females 1
gender = {0 : -1.234323, 1: 0.810161} 
train.GENDER = [gender[item] for item in train.GENDER]
train.GENDER.value_counts()

In [ ]:
biasedlogmodel = LogisticRegression(random_state=101)
biasedlogmodel.fit(X_train,y_train)
logpredictions = biasedlogmodel.predict(X_test)
print(f"Logistic regression validation accuracy: {biasedlogmodel.score(X_test, y_test)}")
print(classification_report(y_test, logpredictions))
confusion_matrix(y_test, logpredictions)

In [ ]:
biasedrfcmodel = RandomForestClassifier(n_estimators=100,max_depth=5)
biasedrfcmodel.fit(X_train, y_train)
rfcpredictions = biasedrfcmodel.predict(X_test)
print(f"Random forest validation accuracy: {biasedrfcmodel.score(X_test, y_test)}")
print(classification_report(y_test, rfcpredictions))
confusion_matrix(y_test, rfcpredictions)

In [ ]:
# Create a Binary Label Dataset to use with AIF360 APIs
train_pp_bld = BinaryLabelDataset(df=pd.concat((X_train, y_train),
                                               axis=1),
                                  label_names=['DEFAULT'],
                                  protected_attribute_names=['GENDER'],
                                  favorable_label=0,
                                  unfavorable_label=1)

privileged_groups = [{'GENDER': 0}]
unprivileged_groups = [{'GENDER': 1}]

In [ ]:
class MetricAdditions:
    def explain(self,
                disp: bool=True) -> Union[None, str]:
        """Explain everything available for the given metric."""

        # Find intersecting methods/attributes between MetricTextExplainer and provided metric.
        inter = set(dir(self)).intersection(set(dir(self.metric)))

        # Ignore private and dunder methods
        metric_methods = [getattr(self, c) for c in inter if c.startswith('_') < 1]

        # Call methods, join to new lines
        s = "\n".join([f() for f in metric_methods if callable(f)])

        if disp:
            print(s)
        else:
            return s  
        
class MetricTextExplainer_(MetricTextExplainer, MetricAdditions):
    """Combine explainer and .explain."""
    pass

In [ ]:
# Create the metric object from the Binary Label Dataset.
metric_train_bld = BinaryLabelDatasetMetric(train_pp_bld,
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
print("# of instances  :", metric_train_bld.num_instances())
print("Base Rate       :", metric_train_bld.base_rate())
print("Consistency     :", metric_train_bld.consistency())
print("Disparate Impact:", metric_train_bld.disparate_impact())
print("Mean Difference :", metric_train_bld.mean_difference())
print("# of negatives(privileged)    :", metric_train_bld.num_negatives(privileged=True))
print("# of negatives(non-privileged):", metric_train_bld.num_negatives(privileged=False))
print("# of positives(privileged)    :", metric_train_bld.num_positives(privileged=True))
print("# of positives(non-privileged):", metric_train_bld.num_positives(privileged=False))
print("Statistical Parity Diference  :", metric_train_bld.statistical_parity_difference()) 
print("")
print("This is the explainer")
# Create the explainer object
explainer = MetricTextExplainer_(metric_train_bld)
# Explain relevant metrics
explainer.explain()

In [ ]:
# Now attempt to mitigate the bias using pre-processing Reweighing, apply to Training dataset
rw = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
# trained model with new weights..
train_pp_bld_f = rw.fit_transform(train_pp_bld)

In [ ]:
# Create Dataframe from Training dataset and new weights and have a look! 
pd.DataFrame({'GENDER': X_train.GENDER,
              'DEFAULT': y_train,
              'Original_weight': np.ones(shape=(X_train.shape[0],)),
              'new_weight': train_pp_bld_f.instance_weights}).sample(20)

In [ ]:
# Use the new Dataframe 
unbiasedlogmodel = LogisticRegression()
unbiasedlogmodel.fit(X_train, y_train, sample_weight=train_pp_bld_f.instance_weights)
print(f"Biased   Logistic regression validation accuracy: {biasedlogmodel.score(X_test,y_test)}")
print(f"Unbiased Logistic regression validation accuracy: {unbiasedlogmodel.score(X_test, y_test)}")

In [ ]:
unbiasedrfcmodel = RandomForestClassifier(n_estimators=100, max_depth=5)
unbiasedrfcmodel.fit(X_train, y_train, sample_weight=train_pp_bld_f.instance_weights)
print(f"Biased   Random forest validation accuracy: {biasedrfcmodel.score(X_test, y_test)}")
print(f"Unbiased Random forest validation accuracy: {unbiasedrfcmodel.score(X_test, y_test)}")

In [ ]:
def plot_auc(y_true: np.ndarray, preds: Dict[str, np.ndarray],
             title: str='', 
             ax=None) -> None:
    leg = []
    for k, p in preds.items():
        fpr, tpr, _ = roc_curve(y_true, p)
        ax = sns.lineplot(x=fpr, 
                          y=tpr,
                          ci=None,
                          ax=ax
                         )
        leg.append(f"{k}: {round(auc(fpr, tpr), 2)}")
   
    ax.legend(leg)
    ax.set_xlabel('FPR')
    ax.set_ylabel('TPR')
    sns.lineplot(x=[0, 1],
                 y=[0, 1],
                 color='gray',
                 ax=ax)
    
    ax.set_title(title)
    
print('Accuracy:')
display(pd.DataFrame({'LogReg': [biasedlogmodel.score(X_test, y_test), 
                                 unbiasedlogmodel.score(X_test, y_test)],
                      'RFC': [biasedrfcmodel.score(X_test, y_test),
                              unbiasedrfcmodel.score(X_test, y_test)]}, 
                     index =['Unfair', 'Fair']))

print('AUC:')
fig, ax = plt.subplots(nrows=1, 
                       ncols=2,
                       figsize=(16, 6))
plot_auc(y_test, 
         {'biased': biasedlogmodel.predict_proba(X_test)[:, 1],
          'unbiased': unbiasedlogmodel.predict_proba(X_test)[:, 1]},
         title='LR',
         ax=ax[0]) 
plot_auc(y_test, 
         {'biased': biasedrfcmodel.predict_proba(X_test)[:, 1],
          'unbiased': unbiasedrfcmodel.predict_proba(X_test)[:, 1]},
         title='RFC',
         ax=ax[1]) 
plt.show()

In [ ]:
def feature_importance(mod: Union[LogisticRegression, RandomForestClassifier],
                       names: List[str],
                       scale=None) -> pd.DataFrame:
    """Return feature importance for LR or RFC models in a sorted DataFrame."""
    if type(mod) == LogisticRegression:
        imp = np.abs(mod.coef_.squeeze()) / scale
        var = np.zeros(shape=imp.shape)
    elif type(mod) == RandomForestClassifier:
        imps = np.array([fi.feature_importances_ for fi in mod.estimators_])
        imp = imps.mean(axis=0)
        var = imps.std(axis=0)

    return pd.DataFrame({'feature': names,
                         'importance': imp,
                         'std': var}).sort_values('importance',
                                                  ascending=False)

def plot_feature_importance(**kwargs) -> None:
    ax = sns.barplot(**kwargs)
    for l in ax.get_xticklabels():
        l.set_rotation(90)

In [ ]:
fig, ax = plt.subplots(nrows=1, 
                       ncols=2,
                       figsize=(16, 6))

plot_feature_importance(x='feature', 
                        y='importance', 
                        data=feature_importance(biasedlogmodel,
                                                names=X_train.columns.tolist(),
                                                scale=X_train.std()),
                       ax=ax[1])
_ = ax[0].set_title('LR Feature Importance - before Mitigating')
plot_feature_importance(x='feature', 
                        y='importance', 
                        data=feature_importance(biasedrfcmodel,
                                                names=X_train.columns.tolist()),
                       ax=ax[0])
_ = ax[1].set_title('RFC Feature Importance - before Mitigating')

In [ ]:
fig, ax = plt.subplots(nrows=1, 
                       ncols=2,
                       figsize=(16, 6))

plot_feature_importance(x='feature', 
                        y='importance', 
                        data=feature_importance(unbiasedlogmodel,
                                                names=X_train.columns.tolist(),
                                                scale=X_train.std()),
                       ax=ax[1])
_ = ax[0].set_title('LR Feature Importance - after Mitigating')
plot_feature_importance(x='feature', 
                        y='importance', 
                        data=feature_importance(unbiasedrfcmodel,
                                                names=X_train.columns.tolist()),
                       ax=ax[0])
_ = ax[1].set_title('RFC Feature Importance - after Mitigating')

In [ ]:
def calc_metrics(mod, x: pd.DataFrame, y_true: pd.DataFrame) -> pd.DataFrame:
    """Calculate fairness metrics at each model threshold."""
    
    # Create a BinaryLabelDataset (as before training)
    bld = BinaryLabelDataset(df=pd.concat((x, y_true),
                                               axis=1),
                                  label_names=['DEFAULT'],
                                  protected_attribute_names=['GENDER'],
                                  favorable_label=0,
                                  unfavorable_label=1)

    privileged_groups = [{'GENDER': 1}]
    unprivileged_groups = [{'GENDER': 2}]
    
    # Create a second set to hold the predicted labels
    bld_preds = bld.copy(deepcopy=True)
    preds = mod.predict_proba(x)[:, 1] # for all the rows, retain the 2nd value.
                                       # preds will contain a single column table of all 
                                       # probabilities that the classification will be 1.
    accuracy = []
    balanced_accuracy = []
    disp_impact = []
    average_abs_odds_difference = []
    avg_odd_diff = []
    equal_opportunity_difference = []
    error_rate = []
    
    
    # For threshold values between 0 and 1:
    thresh = np.linspace(0.01, 0.99, 100) # generate 100 evenly spaced values from 0.01->0.99 
    for t in thresh:
        
        # Apply threshold and set labels in bld for predictions
        bld_preds.labels[preds > t] = 1  # labels is a structured dataset attribute.
        bld_preds.labels[preds <= t] = 0

        # Calculate the metrics for this threshold
        valid_metric = ClassificationMetric(bld, bld_preds, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)

        # Save the balanced accuracy of the model, and the metrics
        accuracy.append(valid_metric.accuracy())
        balanced_accuracy.append(0.5 * (valid_metric.true_positive_rate()
                                        + valid_metric.true_negative_rate()))
        disp_impact.append(np.abs(valid_metric.disparate_impact() - 0.5))
        average_abs_odds_difference.append(valid_metric.average_abs_odds_difference())
        avg_odd_diff.append(valid_metric.average_odds_difference())
        equal_opportunity_difference.append(valid_metric.equal_opportunity_difference())
        error_rate.append(valid_metric.error_rate())

    # Return as df indexed by threshold
    metrics = pd.DataFrame({'accuracy': accuracy,
                            'balanced_accuracy': balanced_accuracy,
                            'disparate_impact': disp_impact,
                            'average_abs_odds_difference': average_abs_odds_difference,
                            'avg_odds_diff': avg_odd_diff,
                            'equal_opportunity_diff': equal_opportunity_difference},
                            index=thresh)
#'error_rate': error_rate},
    return metrics


def plot_metrics(metrics: pd.DataFrame, 
                 title: str='', **kwargs) -> None:
    """Plot the metrics df from calc_metrics with seaborn."""
    ax = sns.lineplot(data=metrics, 
                      **kwargs)
    ax.set_title(title)
    ax.set_xlabel('Classification threshold')

In [ ]:
# Plot for LR
fig, ax = plt.subplots(nrows=1, 
                       ncols=2,
                       figsize=(16, 6))
with warnings.catch_warnings():
    warnings.simplefilter('ignore', RuntimeWarning)
    
    plot_metrics(calc_metrics(biasedlogmodel, X_test, y_test),
                ax=ax[0],
                title="LR: Biased")
    
    plot_metrics(calc_metrics(unbiasedlogmodel, X_test, y_test),
                ax=ax[1],
                title='"LR: Fair"')

In [ ]:
# Plot for RFC
fig, ax = plt.subplots(nrows=1, 
                       ncols=2,
                       figsize=(16, 6))
with warnings.catch_warnings():
    warnings.simplefilter('ignore', RuntimeWarning)
    
    plot_metrics(calc_metrics(biasedrfcmodel, X_test, y_test),
                ax=ax[0],
                title="RFC: Biased")
    
    plot_metrics(calc_metrics(unbiasedrfcmodel, X_test, y_test),
                ax=ax[1],
                title='"RFC: Fair"')